In [35]:
import pandas as pd
df=pd.read_csv("synthetic_customer_loan_data.csv")
df

,customerid,loanno,Month,Loan Amount,EMI Amount,Interest Rate (%),Repayment Period (Months),Credit Score,Credit History (Years),Outstanding Balance Amount,Bank Statement,payment
0,Cus0001,Loan0002,January,273467.56,8976.77,4.23,59,828,18,23774.95,"Transactions: 14, Deposits: 12860.44, Withdraw...",0
1,Cus0001,Loan0002,February,273467.56,8976.77,4.23,59,828,18,23774.95,"Transactions: 14, Deposits: 12860.44, Withdraw...",0
2,Cus0001,Loan0002,March,273467.56,8976.77,4.23,59,828,18,23774.95,"Transactions: 14, Deposits: 12860.44, Withdraw...",0
3,Cus0001,Loan0002,April,273467.56,8976.77,4.23,59,828,18,23774.95,"Transactions: 14, Deposits: 12860.44, Withdraw...",1
4,Cus0001,Loan0002,May,273467.56,8976.77,4.23,59,828,18,23774.95,"Transactions: 14, Deposits: 12860.44, Withdraw...",0
...,...,...,...,...,...,...,...,...,...,...,...,...
3595,Cus0100,Loan0010,August,480896.72,13296.68,6.57,22,834,15,303615.44,"Transactions: 29, Deposits: 83102.3, Withdrawa...",1
3596,Cus0100,Loan0010,September,480896.72,13296.68,6.57,22,834,15,303615.44,"Transactions: 29, Deposits: 83102.3, Withdrawa...",0
3597,Cus0100,Loan0010,October,480896.72,13296.68,6.57,22,834,15,303615.44,"Transactions: 29, Deposits: 83102.3, Withdrawa...",0
3598,Cus0100,Loan0010,November,480896.72,13296.68,6.57,22,834,15,303615.44,"Transactions: 29, Deposits: 83102.3, Withdrawa...",1


In [ ]:
unique_customers_df = df.groupby('Customer Name').agg({
    'Income': 'first',  # Since income remains constant per customer
    'Monthly Expenditure': 'mean',  # Average monthly expenditure
    'Credit Score': 'first'  # Assuming Credit Score is constant per customer
}).reset_index()

# Rename the 'Monthly Expenditure' column to reflect it's an average
unique_customers_df = unique_customers_df.rename(columns={'Monthly Expenditure': 'Average Monthly Expenditure'})

# Display the new DataFrame
print(unique_customers_df)

# Optionally save to a new CSV file
unique_customers_df.to_csv("unique_customer_data.csv", index=False)

In [36]:
unique_customers_df = df.groupby(['customerid', 'loanno']).agg({
    'Loan Amount': 'first',  # First loan amount per customer and loan number
    'Interest Rate (%)': 'first',  # First interest rate per loan
    'EMI Amount': 'first',  # First EMI amount per loan
     'Repayment Period (Months)': 'first',  # First EMI amount per loan
     'Outstanding Balance Amount':'first',
     'Bank Statement':'first',
      'payment': ['sum', lambda x: (x == 0).sum()]  # Sum of 1s (ontime) and count of 0s (offtime)
}).reset_index()

unique_customers_df.columns = ['customerid', 'loanno', 'Loan Amount', 'Interest Rate (%)', 'EMI Amount','Repayment Period (Months)','Outstanding Balance Amount', 'Bank Statement','ontime_payment', 'offtime_payment']

# Display the grouped DataFrame
print(unique_customers_df)
unique_customers_df.to_csv("unique_customerloan_data.csv", index=False)

    customerid    loanno  Loan Amount  Interest Rate (%)  EMI Amount  \
0      Cus0001  Loan0001    246643.85               7.04    14700.49   
1      Cus0001  Loan0002    273467.56               4.23     8976.77   
2      Cus0001  Loan0005    456595.32               7.56    14065.39   
3      Cus0002  Loan0005    491566.43               9.09    13333.74   
4      Cus0002  Loan0006    219856.58               9.81    14391.95   
..         ...       ...          ...                ...         ...   
295    Cus0099  Loan0007    457875.20              10.22     8587.51   
296    Cus0099  Loan0009    271545.03               3.33     9049.74   
297    Cus0100  Loan0004    243283.37              14.64     9226.08   
298    Cus0100  Loan0007    393619.28               3.42    14336.61   
299    Cus0100  Loan0010    480896.72               6.57    13296.68   

     Repayment Period (Months)  Outstanding Balance Amount  \
0                           17                    88540.06   
1          

In [16]:
print(df.columns)

Index(['customerid', 'loanno', 'Month', 'Loan Amount', 'EMI Amount',
       'Interest Rate (%)', 'Repayment Period (Months)', 'Credit Score',
       'Credit History (Years)', 'Outstanding Balance Amount',
       'Bank Statement', 'payment'],
      dtype='object')


In [6]:
import sqlite3 as sq
con=sq.connect("customer.db")
# con.execute("drop table customer")
df.to_sql("customer",con, index=False)

200

In [2]:
import pandas as pd
import numpy as np
import random

# Set a random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Generate unique customer IDs and loan numbers
num_customers = 1000
num_loans_per_customer = 3  # Each customer can have multiple loans
months = [
    "January", "February", "March", "April", "May", 
    "June", "July", "August", "September", "October", 
    "November", "December"
]

# Create a list of loan IDs
loan_ids = [f'Loan{str(i).zfill(4)}' for i in range(1, 11)]

# Create a list to hold the data
data = {
    'customerid': [],
    'loanno': [],
    'Month': [],
    'Loan Amount': [],
    'EMI Amount': [],
    'Interest Rate (%)': [],
    'Repayment Period (Months)': [],
    'Credit Score': [],
    'Credit History (Years)': [],
    'Outstanding Balance Amount': [],
    'Bank Statement': [],
    'payment': []
}

# Generate random data for each customer
for customer_id in range(1, num_customers + 1):
    customer_id_str = f'Cus{str(customer_id).zfill(4)}'
    
    # Randomly select multiple loans for the customer
    selected_loans = random.sample(loan_ids, num_loans_per_customer)

    # Generate data for each selected loan for the same customer
    for loan_id in selected_loans:
        # Generate the fixed attributes for the loan
        loan_amount = round(random.uniform(200000, 500000), 2)  # Random loan amount
        emi_amount = round(random.uniform(8000, 15000), 2)      # Random EMI amount
        interest_rate = round(random.uniform(3.0, 15.0), 2)      # Random interest rate
        repayment_period = random.randint(12, 60)                 # Random repayment period
        credit_score = random.randint(600, 850)                   # Random credit score
        credit_history = random.randint(1, 20)                    # Random credit history years
        outstanding_balance = round(random.uniform(0, loan_amount), 2)  # Random outstanding balance
        bank_statement = f'Transactions: {random.randint(1, 30)}, Deposits: {round(random.uniform(10000, 100000), 2)}, Withdrawals: {round(random.uniform(10000, 100000), 2)}'

        # Generate data for each month for the same customer and loan
        for month in months:
            data['customerid'].append(customer_id_str)
            data['loanno'].append(loan_id)
            data['Month'].append(month)
            data['Loan Amount'].append(loan_amount)
            data['EMI Amount'].append(emi_amount)
            data['Interest Rate (%)'].append(interest_rate)
            data['Repayment Period (Months)'].append(repayment_period)
            data['Credit Score'].append(credit_score)
            data['Credit History (Years)'].append(credit_history)
            data['Outstanding Balance Amount'].append(outstanding_balance)
            data['Bank Statement'].append(bank_statement)
            
            # 90% chance of payment being 1, 10% chance of payment being 0
            payment_status = np.random.choice([1, 0], p=[0.9, 0.1])
            data['payment'].append(payment_status)

# Create DataFrame
df_random = pd.DataFrame(data)

# Display the first few rows of the generated DataFrame
print(df_random.head(12))  # Show first 12 rows to see multiple months for a customer

# Optionally save to a CSV file
df_random.to_csv("synthetic_customer_loan_data.csv", index=False)


import sqlite3 as sq
s=sq.connect("data.db")
df_random.to_sql("customer_loan",s)


   customerid    loanno      Month  Loan Amount  EMI Amount  \
0     Cus0001  Loan0002    January    273467.56     8976.77   
1     Cus0001  Loan0002   February    273467.56     8976.77   
2     Cus0001  Loan0002      March    273467.56     8976.77   
3     Cus0001  Loan0002      April    273467.56     8976.77   
4     Cus0001  Loan0002        May    273467.56     8976.77   
5     Cus0001  Loan0002       June    273467.56     8976.77   
6     Cus0001  Loan0002       July    273467.56     8976.77   
7     Cus0001  Loan0002     August    273467.56     8976.77   
8     Cus0001  Loan0002  September    273467.56     8976.77   
9     Cus0001  Loan0002    October    273467.56     8976.77   
10    Cus0001  Loan0002   November    273467.56     8976.77   
11    Cus0001  Loan0002   December    273467.56     8976.77   

    Interest Rate (%)  Repayment Period (Months)  Credit Score  \
0                4.23                         59           828   
1                4.23                         59

36000

In [5]:
import pandas as pd
import numpy as np
import random

# Set a random seed for reproducibility
random.seed(42)

# Generate unique customer IDs
num_customers = 1000
customer_names = [
    "Andrea Ellis", "Angela Griffin", "Chase Ramos", "Cynthia Washington", 
    "David White", "Emily Johnson", "Frank Wright", "George King", 
    "Hannah Scott", "Isabella Hall", "Jack Turner", "Katherine Adams",
    "Liam Nelson", "Mia Green", "Noah Carter", "Olivia Lee"
]

# Create a list to hold the data
customer_data = {
    'Customerid': [],
    'CustomerName': [],
    'Income': [],
    'AverageMonthlyExpenditure': [],
    'CreditScore': []
}

# Generate random customer data
for customer_id in range(1, num_customers + 1):
    customer_id_str = f'Cus{str(customer_id).zfill(4)}'
    customer_name = random.choice(customer_names)  # Randomly choose a name from the list
    income = round(random.uniform(150000, 200000), 2)  # Random income
    avg_expenditure = round(random.uniform(30000, 60000), 2)  # Random average monthly expenditure
    credit_score = random.randint(300, 850)  # Random credit score

    # Append data to the list
    customer_data['Customerid'].append(customer_id_str)
    customer_data['CustomerName'].append(customer_name)
    customer_data['Income'].append(income)
    customer_data['AverageMonthlyExpenditure'].append(avg_expenditure)
    customer_data['CreditScore'].append(credit_score)

# Create DataFrame
df_customers = pd.DataFrame(customer_data)

# Display the first few rows of the generated DataFrame
print(df_customers.head(10))  # Show first 10 rows to verify data

# Save to a separate CSV file
df_customers.to_csv("customer_data.csv", index=False)


import sqlite3 as sq
s=sq.connect("data.db")
df_customers.to_sql("customer",s)

  Customerid        CustomerName     Income  AverageMonthlyExpenditure  \
0    Cus0001  Cynthia Washington  151250.54                   38250.88   
1    Cus0002         David White  186823.56                   50300.98   
2    Cus0003           Mia Green  151589.13                   32810.86   
3    Cus0004        Andrea Ellis  178062.25                   51480.59   
4    Cus0005         George King  172460.45                   38345.72   
5    Cus0006       Emily Johnson  184906.97                   40207.52   
6    Cus0007        Frank Wright  197860.65                   40097.84   
7    Cus0008         Liam Nelson  154835.82                   55424.83   
8    Cus0009      Angela Griffin  186486.59                   46086.84   
9    Cus0010         Chase Ramos  177602.03                   54882.14   

   CreditScore  
0          528  
1          389  
2          538  
3          729  
4          306  
5          459  
6          394  
7          570  
8          687  
9          670 

1000

In [10]:
import pandas as pd
df=pd.read_csv("synthetic_customer_loan_data.csv")
unique_customers_df = df.groupby(['customerid', 'loanno']).agg({
    'Loan Amount': 'first',  # First loan amount per customer and loan number
    'Interest Rate (%)': 'first',  # First interest rate per loan
    'EMI Amount': 'first',  # First EMI amount per loan
     'Repayment Period (Months)': 'first',  # First EMI amount per loan
     'Outstanding Balance Amount':'first',
     'Bank Statement':'first',
      'payment': ['sum', lambda x: (x == 0).sum()]  # Sum of 1s (ontime) and count of 0s (offtime)
}).reset_index()

unique_customers_df.columns = ['customerid', 'loanno', 'Loan Amount', 'Interest Rate (%)', 'EMI Amount','Repayment Period (Months)','Outstanding Balance Amount', 'Bank Statement','ontime_payment', 'offtime_payment']

# Display the grouped DataFrame
print(unique_customers_df)
unique_customers_df.to_csv("unique_customerloan_data.csv", index=False)

     customerid    loanno  Loan Amount  Interest Rate (%)  EMI Amount  \
0       Cus0001  Loan0001    269798.27               9.73    12214.13   
1       Cus0001  Loan0002    273467.56               4.23     8976.77   
2       Cus0001  Loan0005    201949.63              11.38    13640.73   
3       Cus0002  Loan0002    381117.81              11.76    13649.90   
4       Cus0002  Loan0006    458512.07              11.45    12041.47   
...         ...       ...          ...                ...         ...   
2995    Cus0999  Loan0005    407783.12               8.12     8136.17   
2996    Cus0999  Loan0009    253065.21               8.44    11227.22   
2997    Cus1000  Loan0001    486048.73               6.68     9072.07   
2998    Cus1000  Loan0005    468074.25               3.85    12809.12   
2999    Cus1000  Loan0008    324972.26               8.50    11349.86   

      Repayment Period (Months)  Outstanding Balance Amount  \
0                            57                   113185.72 

In [13]:
import pandas as pd

# Load the data
df1 = pd.read_csv("unique_customerloan_data.csv")

# Group by 'customerid', aggregate the sum for the desired columns, and count the number of loans
result = df1.groupby('customerid').agg({
    'EMI Amount': 'sum',
    'ontime_payment': 'sum',
    'offtime_payment': 'sum',
    'loanno': 'count'  # Count the number of loans per customer
}).reset_index()

# Rename the 'loanno' column to 'loan_count' for clarity
result.rename(columns={'loanno': 'loan_count'}, inplace=True)

# Display the result
print(result)

# Save the result to a new CSV file
result.to_csv("customeremi.csv", index=False)


    customerid  EMI Amount  ontime_payment  offtime_payment  loan_count
0      Cus0001    34831.63              32                4           3
1      Cus0002    36865.24              31                5           3
2      Cus0003    29650.31              35                1           3
3      Cus0004    37619.49              32                4           3
4      Cus0005    28862.84              33                3           3
..         ...         ...             ...              ...         ...
995    Cus0996    43947.45              29                7           3
996    Cus0997    34946.15              29                7           3
997    Cus0998    35864.11              33                3           3
998    Cus0999    30296.99              34                2           3
999    Cus1000    33231.05              28                8           3

[1000 rows x 5 columns]


In [22]:
import pandas as pd

# Load the data
df = pd.read_csv("customeremi.csv")
df1 = pd.read_csv("customer_data.csv")

# Merge the DataFrames on 'customerid'
merged_df = pd.merge(df, df1, left_on='customerid', right_on='Customer ID')

# Drop the duplicate 'Customer ID' column
merged_df.drop(columns=['Customer ID'], inplace=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv("merged_customer_data.csv", index=False)

print("Merged DataFrame saved to 'merged_customer_data.csv'")


Merged DataFrame saved to 'merged_customer_data.csv'


In [23]:
import pandas as pd
df=pd.read_csv("merged_customer_data.csv")

In [24]:
df.head()

,customerid,EMI Amount,ontime_payment,offtime_payment,loan_count,Customer Name,Income,Average Monthly Expenditure,Credit Score
0,Cus0001,34831.63,32,4,3,Cynthia Washington,151250.54,38250.88,528
1,Cus0002,36865.24,31,5,3,David White,186823.56,50300.98,389
2,Cus0003,29650.31,35,1,3,Mia Green,151589.13,32810.86,538
3,Cus0004,37619.49,32,4,3,Andrea Ellis,178062.25,51480.59,729
4,Cus0005,28862.84,33,3,3,George King,172460.45,38345.72,306


In [25]:

# Define loan eligibility criteria
def check_eligibility(row):
    # Criterion 2: Income should be at least 3 times the EMI Amount
    if row['Income'] < 3 * row['EMI Amount']:
        return "Not Eligible"
    
    # Criterion 3: Ontime payments should be at least 80% of total payments
    try:
        total_payments = row['ontime_payment'] + row['offtime_payment']
        if row['ontime_payment'] / total_payments < 0.8:
            return "Not Eligible"
    except:
        pass
    
    # Criterion 4: Customer should not have more than 5 loans
    if row['loan_count'] > 5:
        return "Not Eligible"
    
    # If all criteria are met, customer is eligible
    return "Eligible"

# Apply the eligibility check to each customer
df['Loan Eligibility'] = df.apply(check_eligibility, axis=1)


# # Add total payments column
# df['Total Payments'] = df['ontime_payment'] + df['offtime_payment']

# # Calculate disposable income
# df['Disposable Income'] = df['Income'] - df['Average Monthly Expenditure']

# # Calculate total EMI burden
# df['Total EMI Burden'] = df['EMI Amount'] * df['Total Payments']

# # Calculate eligible loan amount (assuming 40% of disposable income after EMI)
# df['Eligible Loan Amount'] = 0.4 * (df['Disposable Income'] - df['Total EMI Burden'])

# # For customers marked 'Not Eligible', set the eligible loan amount to 0
# df.loc[df['Loan Eligibility'] == 'Not Eligible', 'Eligible Loan Amount'] = 0

# # Display the DataFrame with the new column
# print(df[['customerid', 'Eligible Loan Amount']])
# # Display the DataFrame with eligibility status
# print(df)

# Optionally, save the result to a CSV
df.to_csv("loan_eligibility_data.csv", index=False)

In [26]:
df=pd.read_csv("loan_eligibility_data.csv")
df.head()

,customerid,EMI Amount,ontime_payment,offtime_payment,loan_count,Customer Name,Income,Average Monthly Expenditure,Credit Score,Loan Eligibility
0,Cus0001,34831.63,32,4,3,Cynthia Washington,151250.54,38250.88,528,Eligible
1,Cus0002,36865.24,31,5,3,David White,186823.56,50300.98,389,Eligible
2,Cus0003,29650.31,35,1,3,Mia Green,151589.13,32810.86,538,Eligible
3,Cus0004,37619.49,32,4,3,Andrea Ellis,178062.25,51480.59,729,Eligible
4,Cus0005,28862.84,33,3,3,George King,172460.45,38345.72,306,Eligible


In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras import layers

# Load the dataset
df = pd.read_csv('loan_eligibility_data.csv')

# Drop irrelevant columns (if needed), e.g., Customer Name, customerid
df = df.drop(['customerid', 'Customer Name'], axis=1)

# Encode the categorical target variable 'Loan Eligibility'
le = LabelEncoder()
df['Loan Eligibility'] = le.fit_transform(df['Loan Eligibility'])

# Separate features (X) and target (y)
X = df.drop('Loan Eligibility', axis=1)
y = df['Loan Eligibility']

# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [28]:
# Define the model
model = tf.keras.Sequential()

# Input layer
model.add(layers.InputLayer(input_shape=(X_train.shape[1],)))

# Hidden layers
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))

# Output layer (binary classification: Eligible or Not Eligible)
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                512       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,625
Trainable params: 2,625
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50
25/25 [==============================] - 1s 10ms/step - loss: 0.5873 - accuracy: 0.7588 - val_loss: 0.4105 - val_accuracy: 0.9800
Epoch 2/50
25/25 [==============================] - 0s 3ms/step - loss: 0.3043 - accuracy: 0.9787 - val_loss: 0.2224 - val_accuracy: 0.9800
Epoch 3/50
25/25 [==============================] - 0s 2ms/step - loss: 0.1790 - accuracy: 0.9787 - val_loss: 0.1491 - val_accuracy: 0.9800
Epoch 4/50
25/25 [==============================] - 0s 3ms/step - loss: 0.1293 - accuracy: 0.9787 - val_loss: 0.1186 - val_accuracy: 0.9800
Epoch 5/50
25/25 [==============================] - 0s 3ms/step - loss: 0.1048 - accuracy: 0.9787 - val_loss: 0.1000 - val_accuracy: 0.9800
Epoch 6/50
25/25 [==============================] - 0s 3ms/step - loss: 0.0880 - accuracy: 0.9787 - val_loss: 0.0883 - val_accuracy: 0.9800
Epoch 7/50
25/25 [==============================] - 0s 3ms/step - loss: 0.0760 - accuracy: 0.9787 - val_loss: 0.0785 - val_accuracy: 0.9800
Epoch 8/50
25/25 [=

In [30]:
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)

7/7 [==============================] - 0s 831us/step


In [31]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

7/7 [==============================] - 0s 997us/step - loss: 0.0708 - accuracy: 0.9850
Test Accuracy: 0.99


In [32]:
single_user_data = np.array([[30000, 32, 3, 3, 175000, 40000, 600]])

# Scale the test data using the same scaler used for training data
single_user_data_scaled = scaler.transform(single_user_data)

# Predict using the trained model
prediction = model.predict(single_user_data_scaled)

# Convert the prediction to binary outcome (Eligible or Not Eligible)
predicted_eligibility = 'Eligible' if prediction > 0.5 else 'Not Eligible'

print(f"Loan Eligibility Prediction for the user: {predicted_eligibility}")

1/1 [==============================] - 0s 20ms/step
Loan Eligibility Prediction for the user: Not Eligible


c:\Users\Home\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [33]:
single_user_data = np.array([[34831.63, 32, 4, 3, 151250.54, 38250.88, 528]])

# Scale the test data using the same scaler used for training data
single_user_data_scaled = scaler.transform(single_user_data)

# Predict using the trained model
prediction = model.predict(single_user_data_scaled)

# Convert the prediction to binary outcome (Eligible or Not Eligible)
predicted_eligibility = 'Eligible' if prediction > 0.5 else 'Not Eligible'

print(f"Loan Eligibility Prediction for the user: {predicted_eligibility}")

1/1 [==============================] - 0s 20ms/step
Loan Eligibility Prediction for the user: Not Eligible


c:\Users\Home\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras import layers

# Load the dataset
df = pd.read_csv('loan_eligibility_data.csv')

# Drop irrelevant columns (if needed), e.g., Customer Name, customerid
df = df.drop(['customerid', 'Customer Name'], axis=1)

# Encode the categorical target variable 'Loan Eligibility'
le = LabelEncoder()
df['Loan Eligibility'] = le.fit_transform(df['Loan Eligibility'])

# Separate features (X) and target (y)
X = df.drop('Loan Eligibility', axis=1)
y = df['Loan Eligibility']

# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape the input for LSTM: (samples, time_steps, features)
# Here, we use time_steps = 1 as we're treating each sample as a single timestep
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the LSTM model
model = tf.keras.Sequential()

# LSTM layer
model.add(layers.LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))

# Output layer (binary classification: Eligible or Not Eligible)
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Make predictions
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                18432     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 18,497
Trainable params: 18,497
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
25/25 [==============================] - 2s 20ms/step - loss: 0.6689 - accuracy: 0.8450 - val_loss: 0.6360 - val_accuracy: 0.9800
Epoch 2/50
25/25 [==============================] - 0s 3ms/step - loss: 0.6100 - accuracy: 0.9787 - val_loss: 0.5698 - val_accuracy: 0.9800
Epoch 3/50
25/25 [==============================] - 0s 3ms/step - loss: 0.5412 - accuracy: 0.9787 - val_loss: 0.4925 - val_accuracy: 0.9800
Epoch 4/50
25/25 [=======

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers

# Load the dataset
df = pd.read_csv('loan_eligibility_data.csv')

# Drop irrelevant columns (if needed), e.g., Customer Name, customerid
df = df.drop(['customerid', 'Customer Name'], axis=1)

# Separate features (X) and target (y)
X = df.drop('Loan Eligibility', axis=1).values
y = df['Loan Eligibility'].values

# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape the input for CNN: (samples, height, width, channels)
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1], 1))

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the CNN model
model = tf.keras.Sequential()

# 1D Convolutional layer
model.add(layers.Conv2D(32, (1, 2), activation='relu', input_shape=(1, X_train.shape[2], 1)))
model.add(layers.MaxPooling2D(pool_size=(1, 1)))

# Flatten the output
model.add(layers.Flatten())

# Dense layers
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Make predictions
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 1, 6, 32)          96        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 6, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 192)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                12352     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 12,513
Trainable params: 12,513
Non-trainable params: 0
__________________________________________________

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers

# Load the dataset
df = pd.read_csv('check.csv')



# Separate features (X) and target (y)
X = df.drop('Loan Eligibility', axis=1).values
y = df['Loan Eligibility'].values

# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape the input for CNN: (samples, height, width, channels)
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1], 1))

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the CNN model
model = tf.keras.Sequential()

# 1D Convolutional layer
model.add(layers.Conv2D(32, (1, 2), activation='relu', input_shape=(1, X_train.shape[2], 1)))
model.add(layers.MaxPooling2D(pool_size=(1, 1)))

# Flatten the output
model.add(layers.Flatten())

# Dense layers
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Make predictions
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 1, 6, 32)          96        
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 6, 32)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 192)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                12352     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 12,513
Trainable params: 12,513
Non-trainable params: 0
__________________________________________________

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Load the dataset
df = pd.read_csv('check.csv')

# Separate features (X) and target (y)
X = df.drop('Loan Eligibility', axis=1).values
y = df['Loan Eligibility'].values

# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Create and train the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Test Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(confusion)
print("Classification Report:")
print(report)


Test Accuracy: 1.00
Confusion Matrix:
[[  4   0]
 [  0 196]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00       196

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [2]:
# Sample single test data (adjust the values according to your dataset)
single_test_data = {
    'EMI Amount': 35000.00,                # Example EMI amount
    'ontime_payment': 30,                  # Example on-time payments
    'offtime_payment': 3,                  # Example off-time payments
    'loan_count': 2,                       # Example loan count
    'Income': 180000.00,                   # Example income
    'Average Monthly Expenditure': 50000.00,  # Example expenditure
    'Credit Score': 650                    # Example credit score
}

# Convert to DataFrame
single_test_df = pd.DataFrame([single_test_data])

# Normalize the features using the same scaler used for training
single_test_scaled = scaler.transform(single_test_df)

# Reshape if needed (for 1D input)
single_test_scaled = single_test_scaled.reshape((single_test_scaled.shape[0], -1))

# Make prediction
single_prediction = model.predict(single_test_scaled)

# Convert prediction to eligibility status
eligibility_status = 'Eligible' if single_prediction[0] == 1 else 'Not Eligible'

print(f"Single Test Data Prediction: {eligibility_status}")


Single Test Data Prediction: Eligible


c:\Users\Home\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [41]:
# Sample single test data for "Not Eligible"
not_eligible_test_data = {
    'EMI Amount': 42000.00,                    # Higher EMI amount
    'ontime_payment': 15,                       # Fewer on-time payments
    'offtime_payment': 8,                       # More off-time payments
    'loan_count': 5,                            # Higher number of loans
    'Income': 120000.00,                        # Lower income
    'Average Monthly Expenditure': 70000.00,    # Higher expenditure
    'Credit Score': 450                         # Low credit score
}

# Convert to DataFrame
not_eligible_test_df = pd.DataFrame([not_eligible_test_data])

# Normalize the features using the same scaler used for training
not_eligible_test_scaled = scaler.transform(not_eligible_test_df)

# Reshape if needed (for 1D input)
not_eligible_test_scaled = not_eligible_test_scaled.reshape((not_eligible_test_scaled.shape[0], -1))

# Make prediction
not_eligible_prediction = model.predict(not_eligible_test_scaled)

# Convert prediction to eligibility status
eligibility_status = 'Eligible' if not_eligible_prediction[0] == 1 else 'Not Eligible'

print(f"Single Test Data Prediction: {eligibility_status}")


Single Test Data Prediction: Not Eligible


c:\Users\Home\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [13]:
import pandas as pd
sample=pd.read_csv("sample.csv")
sample


,Customer ID,Customer Name,Income,Average Monthly Expenditure,Credit Score
0,Cus20001,Cynthia Washington,151250.54,38250.88,528


In [14]:
import pandas as pd
sampleloan=pd.read_csv("sampleloan.csv")
sampleloan


,customerid,loanno,Month,Loan Amount,EMI Amount,Interest Rate (%),Repayment Period (Months),Credit Score,Credit History (Years),Outstanding Balance Amount,Bank Statement,payment
0,Cus20001,Loan0002,January,273467.56,8976.77,4.23,59,828,18,23774.95,"Transactions: 14, Deposits: 12860.44, Withdraw...",1
1,Cus20001,Loan0002,February,273467.56,8976.77,4.23,59,828,18,23774.95,"Transactions: 14, Deposits: 12860.44, Withdraw...",0
2,Cus20001,Loan0002,March,273467.56,8976.77,4.23,59,828,18,23774.95,"Transactions: 14, Deposits: 12860.44, Withdraw...",1
3,Cus20001,Loan0002,April,273467.56,8976.77,4.23,59,828,18,23774.95,"Transactions: 14, Deposits: 12860.44, Withdraw...",1
4,Cus20001,Loan0002,May,273467.56,8976.77,4.23,59,828,18,23774.95,"Transactions: 14, Deposits: 12860.44, Withdraw...",1
5,Cus20001,Loan0002,June,273467.56,8976.77,4.23,59,828,18,23774.95,"Transactions: 14, Deposits: 12860.44, Withdraw...",1
6,Cus20001,Loan0002,July,273467.56,8976.77,4.23,59,828,18,23774.95,"Transactions: 14, Deposits: 12860.44, Withdraw...",1
7,Cus20001,Loan0002,August,273467.56,8976.77,4.23,59,828,18,23774.95,"Transactions: 14, Deposits: 12860.44, Withdraw...",1
8,Cus20001,Loan0002,September,273467.56,8976.77,4.23,59,828,18,23774.95,"Transactions: 14, Deposits: 12860.44, Withdraw...",1
9,Cus20001,Loan0002,October,273467.56,8976.77,4.23,59,828,18,23774.95,"Transactions: 14, Deposits: 12860.44, Withdraw...",1


In [15]:
import pandas as pd

unique_customers_df = sampleloan.groupby(['customerid', 'loanno']).agg({
    'Loan Amount': 'first',  # First loan amount per customer and loan number
    'Interest Rate (%)': 'first',  # First interest rate per loan
    'EMI Amount': 'first',  # First EMI amount per loan
     'Repayment Period (Months)': 'first',  # First EMI amount per loan
     'Outstanding Balance Amount':'first',
     'Bank Statement':'first',
      'payment': ['sum', lambda x: (x == 0).sum()]  # Sum of 1s (ontime) and count of 0s (offtime)
}).reset_index()

unique_customers_df.columns = ['customerid', 'loanno', 'Loan Amount', 'Interest Rate (%)', 'EMI Amount','Repayment Period (Months)','Outstanding Balance Amount', 'Bank Statement','ontime_payment', 'offtime_payment']

# Display the grouped DataFrame
print(unique_customers_df)

  customerid    loanno  Loan Amount  Interest Rate (%)  EMI Amount  \
0   Cus20001  Loan0001    269798.27               9.73    12214.13   
1   Cus20001  Loan0002    273467.56               4.23     8976.77   
2   Cus20001  Loan0005    201949.63              11.38    13640.73   

   Repayment Period (Months)  Outstanding Balance Amount  \
0                         57                   113185.72   
1                         59                    23774.95   
2                         33                    43482.53   

                                      Bank Statement  ontime_payment  \
0  Transactions: 15, Deposits: 63033.91, Withdraw...              12   
1  Transactions: 14, Deposits: 12860.44, Withdraw...              10   
2  Transactions: 25, Deposits: 40293.51, Withdraw...              10   

   offtime_payment  
0                0  
1                2  
2                2  


In [16]:
# Group by 'customerid', aggregate the sum for the desired columns, and count the number of loans
result = unique_customers_df.groupby('customerid').agg({
    'EMI Amount': 'sum',
    'ontime_payment': 'sum',
    'offtime_payment': 'sum',
    'loanno': 'count'  # Count the number of loans per customer
}).reset_index()

# Rename the 'loanno' column to 'loan_count' for clarity
result.rename(columns={'loanno': 'loan_count'}, inplace=True)

# Display the result
print(result)

  customerid  EMI Amount  ontime_payment  offtime_payment  loan_count
0   Cus20001    34831.63              32                4           3


In [17]:
# Merge the DataFrames on 'customerid'
merged_df = pd.merge(result, sample, left_on='customerid', right_on='Customer ID')

# Drop the duplicate 'Customer ID' column
merged_df.drop(columns=['Customer ID'], inplace=True)

In [18]:
merged_df



,customerid,EMI Amount,ontime_payment,offtime_payment,loan_count,Customer Name,Income,Average Monthly Expenditure,Credit Score
0,Cus20001,34831.63,32,4,3,Cynthia Washington,151250.54,38250.88,528


In [23]:
data=merged_df.values.tolist()
print(data[0])

print(data[0][6], data[0][1]+data[0][7], 0.3)

['Cus20001', 34831.63, 32, 4, 3, 'Cynthia Washington', 151250.54, 38250.88, 528]
151250.54 73082.51 0.3


In [9]:
scaler = StandardScaler()
X = merged_df.drop(['customerid', 'Customer Name'], axis=1)  # Exclude non-numeric columns for scaling
scaler.fit(X)  # Fit the scaler on the original DataFrame before applying to the test data

# Transform the Not Eligible test data
not_eligible_test_scaled = scaler.transform(X)

# Reshape if needed (for 1D input)
not_eligible_test_scaled = not_eligible_test_scaled.reshape((not_eligible_test_scaled.shape[0], -1))

# Make prediction
not_eligible_prediction = model.predict(not_eligible_test_scaled)
print(not_eligible_prediction)
# Convert prediction to eligibility status
eligibility_status = 'Eligible' if not_eligible_prediction[0] == 1 else 'Not Eligible'

print(f"Single Test Data Prediction: {eligibility_status}")

[1]
Single Test Data Prediction: Eligible


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

# Generate synthetic dataset (replace with real data)
np.random.seed(42)
num_samples = 1000
loan_amount = np.random.randint(100000, 10000000, num_samples)  # Loan amounts between 100k and 10M
interest_rate = np.random.uniform(5, 15, num_samples)           # Interest rate between 5% and 15%
income = np.random.randint(30000, 200000, num_samples)          # Monthly income between 30k and 200k
tenure = np.random.randint(12, 360, num_samples)                # Tenure between 12 months and 30 years
emi = (loan_amount * (interest_rate / 12 / 100) * (1 + interest_rate / 12 / 100) ** tenure) / ((1 + interest_rate / 12 / 100) ** tenure - 1)

# Create DataFrame
data = pd.DataFrame({
    'LoanAmount': loan_amount,
    'InterestRate': interest_rate,
    'Income': income,
    'Tenure': tenure,
    'EMI': emi
})

# Split data into features and target
X = data[['LoanAmount', 'InterestRate', 'Income']]
y_emi = data['EMI']
y_tenure = data['Tenure']

# Train/test split
X_train, X_test, y_emi_train, y_emi_test = train_test_split(X, y_emi, test_size=0.2, random_state=42)
_, _, y_tenure_train, y_tenure_test = train_test_split(X, y_tenure, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train KNN Regressor for EMI prediction
knn_emi = KNeighborsRegressor(n_neighbors=5)
knn_emi.fit(X_train_scaled, y_emi_train)

# Train KNN Regressor for Tenure prediction
knn_tenure = KNeighborsRegressor(n_neighbors=5)
knn_tenure.fit(X_train_scaled, y_tenure_train)

# Function to predict multiple repayment plans
def generate_ai_repayment_options(loan_amount, interest_rate, income):
    # New borrower input (loan details)
    new_data = np.array([[loan_amount, interest_rate, income]])
    new_data_scaled = scaler.transform(new_data)
    
    # Predict EMI and Tenure for the input
    predicted_emi = knn_emi.predict(new_data_scaled)
    predicted_tenure = knn_tenure.predict(new_data_scaled)
    
    # Generate multiple repayment options (vary EMI and tenure)
    tenure_options = np.linspace(max(12, predicted_tenure[0] - 60), min(360, predicted_tenure[0] + 60), 10)
    
    repayment_options = []
    for tenure in tenure_options:
        emi = calculate_emi(loan_amount, interest_rate, tenure)
        repayment_options.append({
            'Tenure (Months)': int(tenure),
            'EMI (INR)': round(emi, 2),
            'Total Interest (INR)': round(emi * tenure - loan_amount, 2),
            'Total Payment (INR)': round(emi * tenure, 2)
        })
    
    return pd.DataFrame(repayment_options)

# Calculate EMI function
def calculate_emi(loan_amount, interest_rate, tenure_months):
    monthly_rate = interest_rate / (12 * 100)
    emi = loan_amount * monthly_rate * (1 + monthly_rate) ** tenure_months / ((1 + monthly_rate) ** tenure_months - 1)
    return emi

# Example input
loan_amount_new = 1000000   # 1 million
interest_rate_new = 7.5     # 7.5% annual interest
income_new = 60000          # Monthly income 60k

# Get multiple repayment options
repayment_options_df = generate_ai_repayment_options(loan_amount_new, interest_rate_new, income_new)

# Display the options
print(repayment_options_df)

   Tenure (Months)  EMI (INR)  Total Interest (INR)  Total Payment (INR)
0               78   16237.66             266537.53           1266537.53
1               91   14402.88             315463.16           1315463.16
2              104   13046.23             365505.83           1365505.83
3              118   12005.55             416654.33           1416654.33
4              131   11184.49             468896.13           1468896.13
5              144   10522.24             522217.35           1522217.35
6              158    9978.50             576602.92           1576602.92
7              171    9525.51             632036.54           1632036.54
8              184    9143.51             688500.88           1688500.88
9              198    8818.07             745977.52           1745977.52


c:\Users\Home\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [10]:
def calculate_loan_amount_without_interest(income_per_month, expenses_per_month, max_emi_ratio=0.3):
    # Calculate net disposable income
    disposable_income = income_per_month - expenses_per_month
    
    # Maximum EMI allowed (as a percentage of net disposable income)
    max_emi = disposable_income * max_emi_ratio
    
    # Eligible loan amount without considering interest (direct multiplication)
    eligible_loan_amount = max_emi * 60
    
    return {
      
        'Eligible Loan Amount (INR)': round(eligible_loan_amount, 2)
    }

# Example borrower profile
borrower_profile = {
    'income_per_month': 60000,   # Monthly income
    'expenses_per_month': 30000,  # Monthly expenses
    'max_emi_ratio': 0.3         # Max 30% of income after expenses for EMI
}



# Calculate eligibility
eligibility = calculate_loan_amount_without_interest(
    borrower_profile['income_per_month'],
    borrower_profile['expenses_per_month'],

    borrower_profile['max_emi_ratio']
)

# Display eligibility details
print("Eligible Loan Details:")
print(eligibility)

Eligible Loan Details:
{'Eligible Loan Amount (INR)': 540000.0}


In [24]:
# Sample data: (CustomerID, LoanNumber, Income, AverageMonthlyExpenditure, CreditScore, LoanAmount, InterestRate, RepaymentPeriodMonths, EMIAmount, OnTimePayments, OffTimePayments)
data = [
    ('Cus20001', 'Loan0001', 151250.54, 38250.88, 528, 269798.27, 9.73, 57, 12214.13, 24, 0),
    ('Cus20001', 'Loan0002', 151250.54, 38250.88, 528, 273467.56, 4.23, 59, 8976.77, 20, 4),
    ('Cus20001', 'Loan0005', 151250.54, 38250.88, 528, 201949.63, 11.38, 33, 13640.73, 20, 4)
]

def calculate_credit_score(loan):
    """
    This function calculates the credit score and total paid amount for a given loan.
    A penalty is applied for off-time payments.
    
    Parameters:
    loan: tuple with loan details
    
    Returns:
    final_credit_score: Credit score after applying penalties
    total_paid: Total amount paid so far (on-time and off-time payments)
    """
    # Extracting relevant values from the loan tuple
    base_score = loan[4]
    on_time_payments = loan[9]
    off_time_payments = loan[10]
    emi_amount = loan[8]

    # Calculate total payments made
    total_paid = (on_time_payments + off_time_payments) * emi_amount

    # Define a penalty for each off-time payment (deduct 10 points per off-time payment)
    penalty = 10  # You can adjust the penalty value here
    
    # Calculate the final credit score after applying penalties for off-time payments
    final_credit_score = base_score - (penalty * off_time_payments)
    
    return final_credit_score, total_paid

# Iterate over the list of loans and calculate results for each one
for loan in data:
    score, total_paid = calculate_credit_score(loan)
    print(f"Loan {loan[1]} - Final Credit Score: {score}, Total Paid: {total_paid:.2f}")


Loan Loan0001 - Final Credit Score: 528, Total Paid: 293139.12
Loan Loan0002 - Final Credit Score: 488, Total Paid: 215442.48
Loan Loan0005 - Final Credit Score: 488, Total Paid: 327377.52
